# Análise comparativa

## 1. Preparação de dados

### 1.1 Configurações iniciais

Importação de bibliotecas

In [2]:
from pathlib import Path
import pandas as pd

Lendo o conjunto de dados

In [3]:
data_path = Path('../data/raw/hour.csv')
df = pd.read_csv(data_path)

Lendo dicionário de dados

In [ ]:
dict_path = Path('../data/external/dicionario.csv')
df_dict = pd.read_csv(dict_path, sep=';')

### 1.2 Tratamento de dados